# Plots of Doppler transform

In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import NarrowbandMetadata
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
import torchsig.utils.dsp as dsp
from torchsig.utils.dsp import (
    torchsig_float_data_type,
    torchsig_complex_data_type
)

import numpy as np
from scipy.signal import find_peaks
from copy import deepcopy
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def generate_tone_signal(num_iq_samples: int = 10, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal.

    """
    sample_rate = 10e6
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['tone'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(torchsig_complex_data_type)

    return signal

In [ ]:
# TODO: provide improved time-scaling interpolator
def doppler(
    data: np.ndarray,
    velocity: float = 1e1,
    propagation_speed: float = 2.9979e8,
    sampling_rate: float = 1.0
) -> np.ndarray:
    """Applies wideband Doppler effect through time scaling.

    Args:
        data (np.ndarray): Complex valued IQ data samples.
        velocity (float): Relative velocity in m/s (positive = approaching). Default 10 m/s.
        propagation_speed (float): Wave speed in medium. Default 2.9979e8 m/s.
        sampling_rate (float): Data sampling rate. Default 1.0.

    Returns:
        np.ndarray: Data with wideband Doppler.

    """
    N = data.size
    
    # calculate time scaling factor
    alpha = propagation_speed / (propagation_speed - velocity)

    # original and scaled signal sample times
    t_orig = np.arange(N) / sampling_rate
    t_new = t_orig * alpha

    # prevent extrapolation beyond original signal duration
    t_new = np.clip(t_new, 0, t_orig[-1])

    # numpy default piece-wise interpolator
    interp_real = np.interp(t_new, t_orig, data.real)
    interp_imag = np.interp(t_new, t_orig, data.imag)
    data = interp_real + 1j*interp_imag
    return data.astype(torchsig_complex_data_type)

In [ ]:
# Doppler test cases
rng = np.random.default_rng(42)

sampling_rate = 1.0
velocity = 1e7
propagation_speed = 2.9979e8

N = 10000
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 
tone_data = tone_bb_data * np.exp(2j * np.pi * 0.2 * np.arange(N) / sampling_rate) # f0 = 0.2

data_d = doppler(
    data = tone_data,
    velocity = velocity,
    propagation_speed = propagation_speed,
    sampling_rate = sampling_rate
)

In [ ]:
# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(tone_data, Fs=sampling_rate, scale='linear', sides='twosided', color='white');
spectrum, freqs, _ = ax.magnitude_spectrum(data_d, Fs=sampling_rate, scale='linear', sides='twosided', color='red');


ax.set_yscale('log')
# ax.set_xscale('log')
# ax.set_xscale('symlog', 
#              base=10,          # Logarithm base
#              linthresh=1/N)   # Threshold for linear region near zero

# Configure axis labels and grid
#ax.set_xlabel('Frequency (Hz) [log]');
ax.set_ylabel('Magnitude [log]');
#plt.ylim([1e-5, None]);

In [ ]:
# alpha_default =  2.9979e8 / ( 2.9979e8 - 1e2)
# print(alpha_default)

alpha_test =  2.9979e8 / ( 2.9979e8 - 1e7)
print(alpha_test)
print("new freq:",alpha_test*0.2)


freqs = np.fft.fftfreq(N) * 4.0
print(freqs[1])


In [ ]:
D = np.abs(np.fft.fft(data_d, norm='ortho'))
freqs = np.fft.fftfreq(N) * sampling_rate

peaks, _ = find_peaks(D, height=0.1, distance=N/20)
plt.plot(freqs, D)
plt.plot(freqs[peaks], D[peaks], "x")

print(freqs[peaks])
#print(D[peaks])
